---
title: "17-广义估计方程(GEE)"
subtitle: "Generalized Estimating Equation"
author: "Simon Zhou"
date: "2025-05-08"
format: 
    html:
        code-fold: false
        fig_caption: true
        number-sections: true
        toc: true
        toc-depth: 2
---

In [1]:
import stata_setup
stata_setup.config('C:/Program Files/Stata18', 'mp', splash=False)

## 广义估计方程

广义估计方程（Generalized Estimating Equations, GEE）是一种用于分析具有相关性数据的统计方法，特别适用于纵向数据和重复测量数据的分析。

### 基本概念

广义估计方程是一种扩展的回归模型，旨在处理因变量之间可能存在的相关性。它于1986年由Liang和Zeger首次提出，主要用于估计广义线性模型的参数，尤其是在数据存在重复测量或相关性时。GEE通过准似然估计方法来处理这些数据，适用于多种类型的因变量，包括连续型、二分类和计数数据等。 

### 应用场景

1. 纵向数据分析：如临床试验中对同一组受试者在不同时间点的测量数据进行分析。
2. 重复测量数据：例如在心理学和社会科学研究中，研究同一对象在不同条件下的表现。
3. 组间比较：用于比较不同组别在某一时间点或多个时间点的差异。 

### 优势与特点

1. 处理相关性：GEE能够有效处理因变量之间的相关性，避免了传统方法（如方差分析）在数据不独立时可能导致的偏差。
2. 灵活性：适用于多种类型的因变量，且对自变量的数据类型没有严格要求，可以处理定类和定量数据。
3. 模型选择：根据因变量的分布特征选择合适的模型，如泊松回归、负二项回归等。 

## 什么是纵向数据分析

Examples oflongitudinal dataanalysis(LDA,纵向数据分析)

1. Child BP measured at each annual visit from 3 to 9 years old
2. Infant weight measured at 3, 6, 9, 12 months
3. In a 3-arm cross-over trial, short chain fatty acid measured at
the end ofeach 

### 纵向数据 vs. 横向数据

纵向数据是指对同一组受试个体或者受试单元在不同时间点上的重复观测若干次，得到由截面和时间序列融合在一起的数据。

因而纵向数据具有自相关性、生态单位聚集性、测量次数与测量时间间隔的非均衡性等特点。

1. Establish temporaltrends(时序:A在B之前发生)
2. Separate cohort effects from aging effects
3. Children have different baseline values in reading abilities
4. The trajectories of reading abilities differs by people

### 方差公式

$$Var(X-Y)
Var((1)X+(-1)Y)=(1^2)Var(X)+(-1^2)Var(Y)+2(1)(-1)Cov(X,Y)$$

在非配对 t 检验中，$Cov(X,Y)$为0，此时 $Var(X-Y)$ 

打破线性相关的假设

## GEE 模型


|correlation | description |
|---|---|
| **exc**hangeable| exchangeable|
| **ind**ependent| independent|
| **uns**tructured| unstructured|
| **fix**ed matname| user-specified|
| ar #| autoregressive of order #|
| **sta**tionary #| stationary of order #|
| **non**stationary #| nonstationary of order #|

### GEE 常用步骤

1. 使用 independent 回归结构
2. 稳健标准误估计（robust variance estimate）；如果用错模型，稳健后的结果也会相差不远
3. 使用 QIC（AIC for GEE）评估GEE模型

## 数据导入

Data:**NLS Women 14-24 in 1968**

In [2]:
%%stata
webuse union,clear

(NLS Women 14-24 in 1968)


## 数据观测

In [4]:
%%stata
list in 1/10


     +----------------------------------------------------------------+
     | idcode   year   age   grade   not_smsa   south   union   black |
     |----------------------------------------------------------------|
  1. |      1     72    20      12          0       0       1       1 |
  2. |      1     77    25      12          0       0       0       1 |
  3. |      1     80    28      12          0       0       1       1 |
  4. |      1     83    31      12          0       0       1       1 |
  5. |      1     85    33      12          0       0       1       1 |
     |----------------------------------------------------------------|
  6. |      1     87    35      12          0       0       1       1 |
  7. |      1     88    37      12          0       0       1       1 |
  8. |      2     71    19      12          0       0       0       1 |
  9. |      2     77    25      12          0       0       1       1 |
 10. |      2     78    26      12          0       0       1  

## 用法

### GEE 准备

```stata
xtset studyid timevar
```

- `studyid`: unique ID for each participant
- `timevar`: timevar will usually be a variable that counts 1,2,..., and is to be interpreted as firstyear ofsurvey, second year,..., or first month oftreatment, second month.

In [5]:
%%stata
xtset id year


Panel variable: idcode (unbalanced)
 Time variable: year, 70 to 88, but with gaps
         Delta: 1 unit


### GEE 命令

```stata
xtgee y x_1,x_2,x_3,…[if] [in] [weight] ,family(family) link(link) corr(correlation structure) robust
```

可以使用 `help gee` 查看更多信息

In [6]:
%%stata
xtgee union age grade not_smsa south, family(binomial) link(logit) corr(ind) robust


Iteration 1:  Tolerance = 1.940e-12

GEE population-averaged model                      Number of obs    =   26,200
Group variable: idcode                             Number of groups =    4,434
Family: Binomial                                   Obs per group:  
Link:   Logit                                                   min =        1
Correlation: independent                                        avg =      5.9
                                                                max =       12
                                                   Wald chi2(4)     =   160.30
Scale parameter = 1                                Prob > chi2      =   0.0000

Pearson chi2(26200)  = 26242.04                    Deviance         = 27093.19
Dispersion (Pearson) = 1.001604                    Dispersion       = 1.034091

                                 (Std. err. adjusted for clustering on idcode)
------------------------------------------------------------------------------
             |         